In [156]:
!pip install statsmodels

     |████████████████████████████████| 9.5 MB 496 kB/s eta 0:00:01
  Using cached patsy-0.5.1-py2.py3-none-any.whl (231 kB)
     |████████████████████████████████| 30.7 MB 32.1 MB/s eta 0:00:01


In [15]:
import pandas as pd
from pathlib import Path
import plotly.express as px

import json

In [37]:
with open('states_hash.json') as f:
    states = json.load(f)
    
all_states = list(states.values())


    
def filter_on_state(df):
    df = df[df['State'].isin(all_states)]
    return df

def preprocess(df):
    df = df.loc[:60]
    df.rename({'Census Division\nand State': "State"}, axis=1, inplace=True)
    df = filter_on_state(df)
    return df


In [111]:
df_gas = preprocess(pd.read_excel(Path('/Users/archydeberker/Downloads/table_1_07_b.xlsx'), header=5))
df_wind = preprocess(pd.read_excel(Path('/Users/archydeberker/Downloads/table_1_14_b.xlsx'), header=5))

/Users/archydeberker/Desktop/code/good-news-graphs/analysis/gng/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
df_r = preprocess(pd.read_excel(Path('/Users/archydeberker/Downloads/table_1_11_b.xlsx'), header=5))

/Users/archydeberker/Desktop/code/good-news-graphs/analysis/gng/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
df = df_gas.set_index('State')[['November 2020 YTD']].join(df_.set_index('State')['November 2020 YTD'], rsuffix='_renewables')

In [53]:
px.scatter(df, x='November 2020 YTD', y='November 2020 YTD_renewables', hover_name=df.index)

In [78]:
pop = pd.read_excel('/Users/archydeberker/Downloads/nst-est2019-01.xlsx', header=3)
pop.rename({'Unnamed: 0': 'State'}, axis=1, inplace=True)
pop['State'] = pop['State'].str.lstrip('.')
pop.head(10)

,State,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.0,308758105.0,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
1,Northeast,55317240.0,55318443.0,55380134.0,55604223.0,55775216.0,55901806.0,56006011.0,56034684.0,56042330.0,56059240.0,56046620.0,55982803.0
2,Midwest,66927001.0,66929725.0,66974416.0,67157800.0,67336743.0,67560379.0,67745167.0,67860583.0,67987540.0,68126781.0,68236628.0,68329004.0
3,South,114555744.0,114563030.0,114866680.0,116006522.0,117241208.0,118364400.0,119624037.0,120997341.0,122351760.0,123542189.0,124569433.0,125580448.0
4,West,71945553.0,71946907.0,72100436.0,72788329.0,73477823.0,74167130.0,74925793.0,75742555.0,76559681.0,77257329.0,77834820.0,78347268.0
5,Alabama,4779736.0,4780125.0,4785437.0,4799069.0,4815588.0,4830081.0,4841799.0,4852347.0,4863525.0,4874486.0,4887681.0,4903185.0
6,Alaska,710231.0,710249.0,713910.0,722128.0,730443.0,737068.0,736283.0,737498.0,741456.0,739700.0,735139.0,731545.0
7,Arizona,6392017.0,6392288.0,6407172.0,6472643.0,6554978.0,6632764.0,6730413.0,6829676.0,6941072.0,7044008.0,7158024.0,7278717.0
8,Arkansas,2915918.0,2916031.0,2921964.0,2940667.0,2952164.0,2959400.0,2967392.0,2978048.0,2989918.0,3001345.0,3009733.0,3017804.0
9,California,37253956.0,37254519.0,37319502.0,37638369.0,37948800.0,38260787.0,38596972.0,38918045.0,39167117.0,39358497.0,39461588.0,39512223.0


In [80]:
df = df.join(pop.set_index('State')[[2019]], how='left').rename({2019: 'Population 2019'}, axis=1)

In [81]:
px.scatter(df, x='November 2020 YTD', y='November 2020 YTD_renewables', hover_name=df.index, size='Population 2019')

In [86]:
df_outages = pd.read_excel('~/Downloads/table_b_1.xlsx', header=1)

In [276]:
df_outages.groupby('NERC Region').sum()


,Month
NERC Region,
FRCC,14.0
MRO,62.0
NPCC,153.0
RF,225.0
SERC,298.0
SPP RE,57.0
TRE,199.0
WECC,424.0


In [136]:
def count_outages(df_outages):
    df_outages.dropna(subset=['Area Affected'], inplace=True, axis=0)
    outage_dict = {state: {'Count': 0, 'People': 0} for state in all_states}

    for _, row in df_outages.iterrows():
        all_geographies = [area.strip(':') for area in row['Area Affected'].split(' ')]
        for geo in all_geographies:
            if geo in outage_dict:
                outage_dict[geo]['Count'] +=1 
                try:
                    num = int(row['Number of Customers Affected'])
                    outage_dict[geo]['People'] += num
                except:
                    pass
                
        
    return outage_dict

In [137]:
out_dict = count_outages(df_outages)

In [138]:
out_dict

{'Alabama': {'Count': 10, 'People': 596074},
 'Alaska': {'Count': 0, 'People': 0},
 'American Samoa': {'Count': 0, 'People': 0},
 'Arizona': {'Count': 2, 'People': 0},
 'Arkansas': {'Count': 10, 'People': 823147},
 'California': {'Count': 33, 'People': 1975141},
 'Colorado': {'Count': 4, 'People': 13000},
 'Connecticut': {'Count': 9, 'People': 2011803},
 'Delaware': {'Count': 4, 'People': 1023110},
 'District Of Columbia': {'Count': 0, 'People': 0},
 'Federated States Of Micronesia': {'Count': 0, 'People': 0},
 'Florida': {'Count': 7, 'People': 339077},
 'Georgia': {'Count': 8, 'People': 428474},
 'Guam': {'Count': 0, 'People': 0},
 'Hawaii': {'Count': 0, 'People': 0},
 'Idaho': {'Count': 1, 'People': 0},
 'Illinois': {'Count': 5, 'People': 1291000},
 'Indiana': {'Count': 4, 'People': 349122},
 'Iowa': {'Count': 4, 'People': 550000},
 'Kansas': {'Count': 0, 'People': 0},
 'Kentucky': {'Count': 2, 'People': 164136},
 'Louisiana': {'Count': 16, 'People': 2292937},
 'Maine': {'Count': 10,

In [144]:
outages_count_df = pd.DataFrame.from_dict(out_dict).T

In [145]:
outages_count_df

,Count,People
Alabama,10,596074
Alaska,0,0
American Samoa,0,0
Arizona,2,0
Arkansas,10,823147
California,33,1975141
Colorado,4,13000
Connecticut,9,2011803
Delaware,4,1023110
District Of Columbia,0,0


In [146]:
df = df.join(outages_count_df)

In [108]:
px.scatter(df, x='November 2020 YTD', y='November 2020 YTD_renewables', hover_name=df.index, size='Population 2019',
          color='Number Outages 2020')

In [147]:
px.scatter(df, x='November 2020 YTD_renewables', y='Number Outages 2020', hover_name=df.index, size='People',
          color='Number Outages 2020')

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['November 2020 YTD', 'November 2020 YTD_Wind', 'Population 2019', 'Number Outages 2020', 'Count', 'People'] but received: November 2020 YTD_renewables

In [110]:
px.scatter(df, x='November 2020 YTD', y='Number Outages 2020', hover_name=df.index, size='Population 2019',
          color='Number Outages 2020')

In [214]:
df = df_gas.set_index('State')[['November 2020 YTD']].join(df_wind.set_index('State')['November 2020 YTD'], rsuffix='_Wind', lsuffix='_Gas')
df = df.join(df_r.set_index('State')['November 2020 YTD']).rename({'November 2020 YTD': 'November 2020 YTD_Renewables'},axis=1)
df = df.join(pop.set_index('State')[[2019]], how='left').rename({2019: 'Population 2019'}, axis=1)
df = df.join(outages_count_df).rename({'Count': 'Number Outages 2020',
                                      'People': 'People affected by Outages 2020'}, axis=1)
df.head(5)

,November 2020 YTD_Gas,November 2020 YTD_Wind,November 2020 YTD_Renewables,Population 2019,Number Outages 2020,People affected by Outages 2020
State,,,,,,
Connecticut,21440.0,13.0,912.0,3565287.0,9,2011803
Maine,1614.0,2357.0,4288.0,1344212.0,10,2351803
Massachusetts,12709.0,242.0,2585.0,6892503.0,9,2011803
New Hampshire,3531.0,521.0,1399.0,1359711.0,0,0
Rhode Island,6814.0,212.0,568.0,1059361.0,0,0


In [215]:
px.scatter(df, x='November 2020 YTD_Wind', y='Number Outages 2020', hover_name=df.index, size='Population 2019',
          color='Number Outages 2020', log_x=True)

In [229]:
px.scatter(df, x='November 2020 YTD_Gas', y='Number Outages 2020', 
           hover_name=df.index,
           size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [230]:
px.scatter(df, x='November 2020 YTD_Wind', y='Number Outages 2020', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [180]:
df_total_gen = preprocess(pd.read_excel('/Users/archydeberker/Downloads/table_1_03_b.xlsx', header=5))

/Users/archydeberker/Desktop/code/good-news-graphs/analysis/gng/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [217]:
df_total_gen.rename({'November 2020 YTD': 'Total MWh'}, axis=1, inplace=True)
df = df.join(df_total_gen.set_index('State')['Total MWh'])

In [277]:
df['% Wind'] = df['November 2020 YTD_Wind'] / df['Total MWh'] * 100
df['% Gas'] = df['November 2020 YTD_Gas'] / df['Total MWh']* 100
df['% Renewable'] = df['November 2020 YTD_Renewables'] / df['Total MWh']* 100

df['% Pop Affected']  = df['People affected by Outages 2020'] / df['Population 2019'] * 100
df['Outages per person']  = df['Number Outages 2020'] / df['Population 2019'] 

In [281]:
fig = px.scatter(df, x='% Gas', y='Number Outages 2020', hover_name=df.index,
           size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols',
                template='plotly_white')
fig.show()
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     5.096
Date:                Thu, 18 Feb 2021   Prob (F-statistic):             0.0286
Time:                        16:40:13   Log-Likelihood:                -164.64
No. Observations:                  50   AIC:                             333.3
Df Residuals:                      48   BIC:                             337.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1638      1.641      1.319      0.194      -1.135       5.463
x1             0.0868      0.038      2.258      0.029       0.009       0.164
==============================================================================
Omnibus:                       43.494   Durbin-Watson:                   1.838
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              149.380
Skew:                           2.376   Prob(JB):                     3.65e-33
Kurtosis:                      10.008   Cond. No.                         74.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [271]:
fig = px.scatter(df, x='% Gas', y='People affected by Outages 2020', hover_name=df.index,
           size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     8.070
Date:                Thu, 18 Feb 2021   Prob (F-statistic):            0.00658
Time:                        16:28:26   Log-Likelihood:                -755.00
No. Observations:                  50   AIC:                             1514.
Df Residuals:                      48   BIC:                             1518.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.356e+05    2.2e+05      0.616      0.541   -3.07e+05    5.79e+05
x1          1.466e+04   5162.020      2.841      0.007    4285.627     2.5e+04
==============================================================================
Omnibus:                       25.567   Durbin-Watson:                   1.535
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.683
Skew:                           1.667   Prob(JB):                     8.89e-10
Kurtosis:                       5.983   Cond. No.                         74.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [267]:
results['px_fit_results'].value

AttributeError: 'Series' object has no attribute 'value'

In [221]:
px.scatter(df, x='% Wind', y='People affected by Outages 2020', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [223]:
px.scatter(df, x='% Renewable', y='People affected by Outages 2020', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [228]:
px.scatter(df, x='% Renewable', y='% Pop Affected', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols', log_y=True)

In [261]:
px.scatter(df, x='% Gas', y='Number Outages 2020', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [262]:
px.scatter(df, x='% Renewable', y='Number Outages 2020', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols')

In [260]:
px.scatter(df, x='% Gas', y='% Pop Affected', hover_name=df.index, size='People affected by Outages 2020',
          color='Number Outages 2020', trendline='ols', log_y=True)

In [231]:
df_temp = pd.read_csv('/Users/archydeberker/Downloads/avg_winter_temp.csv')

In [233]:
df_temp = df_temp.drop('Unnamed: 0', axis=1).rename({'Unnamed: 1': 'State'}, axis=1).set_index('State')

In [234]:
df = df.join(df_temp)

In [243]:
px.scatter(df, x='% Renewable', color='Min Temp C', y='People affected by Outages 2020', size='People affected by Outages 2020',
          hover_name=df.index)

In [246]:
px.scatter(df, x='% Gas', color='Min Temp C', y='% Renewable', size='People affected by Outages 2020',
          hover_name=df.index)

In [ ]:
px.scatter(df, x='% Gas', color='Min Temp C', y='% Renewable', size='People affected by Outages 2020',
          hover_name=df.index)

In [248]:
df['Colder than freezing'] = df['Min Temp C'] < 0

In [259]:
px.scatter(df, x='% Gas', color='Colder than freezing', size='People affected by Outages 2020',
           y='Number Outages 2020',
          hover_name=df.index)

In [258]:
px.scatter(df, x='November 2020 YTD_Wind', color='Colder than freezing', size='People affected by Outages 2020',
           y='Number Outages 2020',
          hover_name=df.index)

In [294]:
df.index.isin(['California', 'Texas'])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
        True, False, False, False, False])

In [296]:
fig = px.scatter(df.loc[~df.index.isin(['California', 'Texas'])], 
                 x='% Gas', 
                 y='Number Outages 2020',
                 hover_name=df.loc[~df.index.isin(['California', 'Texas'])].index, 
                 size='Population 2019',
              color='Number Outages 2020',
                 trendline='ols')
fig.show()
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     6.706
Date:                Thu, 18 Feb 2021   Prob (F-statistic):             0.0128
Time:                        16:46:20   Log-Likelihood:                -133.75
No. Observations:                  48   AIC:                             271.5
Df Residuals:                      46   BIC:                             275.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0024      0.990      2.022      0.049       0.009       3.996
x1             0.0606      0.023      2.590      0.013       0.013       0.108
==============================================================================
Omnibus:                        4.870   Durbin-Watson:                   1.728
Prob(Omnibus):                  0.088   Jarque-Bera (JB):                4.244
Skew:                           0.727   Prob(JB):                        0.120
Kurtosis:                       3.078   Cond. No.                         72.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [290]:
fig = px.scatter(df, 
                 x='% Renewable', 
                 y='Number Outages 2020',
                 hover_name=df.index, 
                 size='Population 2019',
              color='Number Outages 2020',
                 trendline='ols')
fig.show()
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.5648
Date:                Thu, 18 Feb 2021   Prob (F-statistic):              0.456
Time:                        16:44:27   Log-Likelihood:                -166.87
No. Observations:                  50   AIC:                             337.7
Df Residuals:                      48   BIC:                             341.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4226      1.427      3.099      0.003       1.554       7.292
x1             0.0543      0.072      0.752      0.456      -0.091       0.200
==============================================================================
Omnibus:                       40.643   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.389
Skew:                           2.282   Prob(JB):                     7.21e-27
Kurtosis:                       9.080   Cond. No.                         28.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [319]:
min_temps = pd.read_csv('/Users/archydeberker/Downloads/min_temps.csv')
min_temps.rename({'Temp.\n°C': 'Temp Celsius'}, axis=1, inplace=True)

In [333]:
df.drop('Temp Celsius', axis=1, inplace=True)
df = df.join(min_temps.set_index('State')['Temp Celsius'].apply(lambda x: -float(x.strip('–'))))

In [451]:

df['label'] = ''
for state in ['California', 'Texas', 'Iowa', 'North Dakota',  'Kansas']:
    df.loc[state, 'label']  = f"<em>{state}</em><br> {df.loc[state, 'Avg Winter °C']} °C"

In [454]:
df.rename({'Min Temp C': 'Avg Winter °C'}, inplace=True, axis=1)
fig = px.scatter(df, x='% Wind', y='Number Outages 2020', color='Avg Winter °C', hover_name=df.index,
          size='Population 2019', template='plotly_white', text='label',
                color_continuous_scale='RdBu_r')

fig.update_traces( textfont=dict(
        size=15,

    ))
fig.update_traces(textposition='top center')
fig.update_layout(font_family='Futura',
    title=dict(text='Many states with cold winters rely heavily upon wind with few outages',
              xanchor='center', x=.5, font=dict(size=22)),
                 yaxis=dict(range=(-4,40), title='Number of Outages in 2020'),
                 xaxis=dict(title=dict(text='% Wind Electricity Generation',
                           font=dict(size=16,))))

fig.add_annotation(x=1,y=1, text='<i> Bubbles sized by Population </i>', xref='paper', yref='paper', showarrow=False,
                  font=dict(size=12, color='rgb(200,200,200)'))


fig.add_annotation(x=0,y=-.15, text='<i> Data: EIA </i>', xref='paper', yref='paper', showarrow=False,
                  font=dict(size=12, color='rgb(200,200,200)'))

In [440]:
df.rename({'Min Temp C': 'Avg Winter °C'}, inplace=True, axis=1)
fig = px.scatter(df, x='% Gas', y='Number Outages 2020', color='Avg Winter °C', hover_name=df.index,
          size='Population 2019', template='plotly_white', text='label',
                color_continuous_scale='RdBu_r')

fig.update_traces( textfont=dict(
        size=15,

    ))
fig.update_traces(textposition='top center')
fig.update_layout(font_family='Futura',
    title=dict(text='Texas has many more outages than states which are colder and higher % renewable generation',
              xanchor='center', x=.5, font=dict(size=20)),
                 yaxis=dict(range=(-4,40), title='Number of Outages in 2020'),
                 xaxis=dict(title=dict(text='% Gas Electricity Generation',
                           font=dict(size=16,))))

fig.add_annotation(x=1,y=1, text='<i> Bubbles sized by Population </i>', xref='paper', yref='paper', showarrow=False,
                  font=dict(size=12, color='rgb(200,200,200)'))


fig.add_annotation(x=0,y=-.15, text='<i> Data: EIA </i>', xref='paper', yref='paper', showarrow=False,
                  font=dict(size=12, color='rgb(200,200,200)'))